## Import the module

In [ ]:
import pandas as pd

## Load the csv file into pandas dataframe

In [ ]:
data = pd.read_csv(r"C:\Users\Welcome\Downloads\HousingDataset (2).csv", engine='python')
data.head()

# Section 1

## Environment setup and data cleaning

In [ ]:
# shape: Return a tuple representing the dimensionality of the DataFrame.
data.shape

In [ ]:
# info: Print a concise summary of a DataFrame.
data.info()

In [ ]:
# describe: Generate descriptive statistics.
data.describe()

In [ ]:
# isnull(): Detect missing values.
data.isnull().sum()

### 1.1 Identify and remove the following columns:
a. ad_type
b. title
c. description
d. l4
e. l5
f. l6

In [ ]:
# As instructed we are dropping the columns 'ad_type', 'title', 'description', 'l4', 'l5', and 'l6'
# Drop: will drop specified labels from rows or columns.
data1 = data.drop(['ad_type','title','description','l4','l5','l6'],axis =1) 
data1.shape

### 1.2 Investigate and discard all rows that contain Null values in any of these fields:
a. lon
b. lat
c. price_period
d. bedrooms
e. surface_total
f. rooms
g. price
h. surface_covered

In [ ]:
# Counting the null values in each column of the DataFrame.
data1.isnull().sum()

In [ ]:
# we will remove null values using dropna function
data2 = data1.dropna(subset=['lon','lat','price_period','bedrooms','surface_total','rooms','price', 'surface_covered'])
data2

In [ ]:
data2.isnull().sum()

### 1.3 Separating the DF into 2 Tables, One with property_details and another with property_price_details 

In [ ]:
property_details_df = data2[['id', 'start_date', 'end_date', 'created_on', 'lat', 'lon', 'l1',
                         'l2', 'l3', 'rooms', 'bedrooms', 'bathrooms','surface_total', 'surface_covered'] ]
property_details_df.head()

In [ ]:
property_price_details_df = data2[['id', 'price', 'currency', 'price_period', 'property_type', 'operation_type']]
property_price_details_df.head()

In [ ]:
# pip install pandas sqlalchemy sqlite3

In [ ]:
# creating a sqlite database 
from sqlalchemy import create_engine
db_path = 'sqlite:///HousingData.db'
engine = create_engine(db_path)

In [ ]:
# converting dataframe into sql table
property_details_df.to_sql('Property_Details', engine, if_exists='replace', index=False)
property_price_details_df.to_sql('Property_Price_Details', engine, if_exists='replace', index=False)

In [ ]:
# To understand better we are printing the schema of both the tables
db_path = 'sqlite:///HousingData.db'
engine = create_engine(db_path)

property_details_schema_query = "PRAGMA table_info(Property_Details)"
property_details_schema_result = engine.execute(property_details_schema_query)

print("Schema for Property_Details:")
for row in property_details_schema_result:
    print(row)

property_price_details_schema_query = "PRAGMA table_info(Property_Price_Details)"
property_price_details_schema_result = engine.execute(property_price_details_schema_query)

print("\nSchema for Property_Price_Details:")
for row in property_price_details_schema_result:
    print(row)

# Section 2

## Data Analysis

#### • Retrieve properties that have a price greater than 1 million and are located in "Estados Unidos" (l1).

In [ ]:
query = """
    SELECT PD.*
FROM Property_Details PD
JOIN Property_Price_Details PPD ON PD.id = PPD.id
WHERE PPD.price > 1000000 AND PD.l1 = 'Estados Unidos';
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)  
    # results.keys() is a method to extract the column names from query result
    df1 = pd.DataFrame(results.fetchall(), columns=results.keys()) # to retrieve all rows of data from the result set.
    
# Print the resulting DataFrame
print(df1)

#### • Categorize properties based on their surface area as 'Small' if it's less than 50 square meters, 'Medium' if it's between 50 and 100 square meters, and 'Large' if it's greater than 100 square meters

In [ ]:
query = """
SELECT *,
    CASE
        WHEN surface_total < 50 THEN 'Small'
        WHEN surface_total >= 50 AND surface_total <= 100 THEN 'Medium'
        WHEN surface_total > 100 THEN 'Large'
        ELSE 'Unknown'
    END AS surface_area_category
FROM Property_Details;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df2 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df2)

#### • List all properties (id) in the "Belgrano" neighborhood (l3) that have the same number of bedrooms and bathrooms as another property in the dataset

In [ ]:
query = """
    SELECT DISTINCT P1.id
    FROM Property_Details P1
    JOIN Property_Details P2
    ON P1.bedrooms = P2.bedrooms
        AND P1.bathrooms = P2.bathrooms
        AND P1.id <> P2.id
    WHERE P1.l3 = 'Belgrano';
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df3 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df3)

#### • Calculate the average price per square meter (price / surface_total) for each property type (property_type) in the "Belgrano" neighborhood (l3)

In [ ]:
query = """
    SELECT property_type,
       AVG(price / surface_total) AS avg_price_per_sq_meter
FROM Property_Details PD
JOIN Property_Price_Details PPD ON PD.id = PPD.id
WHERE PD.l3 = 'Belgrano'
GROUP BY property_type;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df4 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df4)

#### • Identify properties that have a higher price than the average price of properties with the same number of bedrooms and bathrooms.

In [ ]:
query = """
    SELECT pd.id, pd.bedrooms, pd.bathrooms, pp.price
FROM Property_Details pd
INNER JOIN Property_Price_Details pp ON pd.id = pp.id
WHERE pp.price > (
    SELECT AVG(price) 
    FROM Property_Price_Details 
    WHERE bedrooms = pd.bedrooms 
    AND bathrooms = pd.bathrooms
) LIMIT 50;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df5 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print("Since the output is so huge we are restricting to 500 rows")
print(df5)

#### • Calculate the cumulative price for each property type, ordered by the creation date.

In [ ]:
query = """
    SELECT
    created_on,
    property_type,
    price,
    SUM(price) OVER (PARTITION BY property_type ORDER BY created_on) AS cumulative_price
FROM Property_Details PD
JOIN Property_Price_Details PPD ON PD.id = PPD.id
ORDER BY created_on;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df6 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df6)

#### • Identify the 10 locations (l3) with the highest total surface area (sum of surface_total) of properties listed for sale (operation_type = 'Venta'):

In [ ]:
query = """
    SELECT l3,
       SUM(surface_total) AS total_surface_area
FROM Property_Details PD
JOIN Property_Price_Details PPD ON PD.id = PPD.id
WHERE PPD.operation_type = 'Venta'
GROUP BY l3
ORDER BY total_surface_area DESC
LIMIT 10;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df7 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df7)

#### • Find the top 5 most expensive properties (based on price) in the "Palermo" neighborhood (l3) that were listed in August 2020:

In [ ]:
query = """
    SELECT PD.id,
       PD.id,
       PD.l3,
       PPD.price,
       PD.created_on
FROM Property_Details PD
JOIN Property_Price_Details PPD ON PD.id = PPD.id
WHERE PD.l3 = 'Palermo' 
  AND strftime('%Y-%m', PD.start_date) = '2020-08'
ORDER BY PPD.price DESC
LIMIT 5;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df8 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df8)

#### • Find the top 3 properties with the highest price per square meter (price divided by surface area) within each property type.

In [ ]:
query = """
SELECT property_type, 
       ppd.id AS property_id, 
       MAX(price) AS price, 
       MAX(surface_total) AS surface_total, 
       MAX(price / surface_total) AS price_per_square_meter
       FROM Property_Price_Details ppd
       JOIN Property_Details pd ON ppd.id = pd.id
       GROUP BY property_type, property_id
       ORDER BY price_per_square_meter DESC
       LIMIT 3;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df9 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df9)

#### • Find the top 3 locations (l1, l2, l3) with the highest average price per square meter (price / surface_total) for properties listed for sale (operation_type = 'Venta') in the year 2020. Exclude locations with fewer than 10 properties listed for sale in 2020 from the results.

In [ ]:
query = """
SELECT  l1, 
        l2,
        l3, 
        AVG(price / surface_total) AS average_price_per_square_meter 
        FROM Property_Price_Details ppd
        JOIN Property_Details pd ON ppd.id = pd.id 
        WHERE operation_type = 'Venta' 
        AND start_date >= '2020-01-01' AND start_date <= '2020-12-31' 
        GROUP BY l1, l2, l3 HAVING COUNT(*) >= 10 
        ORDER BY average_price_per_square_meter DESC 
        LIMIT 3;
"""

# Execute the query and retrieve the results into a DataFrame
with engine.connect() as connection:
    results = connection.execute(query)
    df10 = pd.DataFrame(results.fetchall(), columns=results.keys())

# Print the resulting DataFrame
print(df10)

# Section 3 

## Expose the results in API 

In [ ]:
# pip install flask,waitress

In [ ]:
from flask import Flask, Response, jsonify
import pandas as pd
from waitress import serve
import os,signal

app = Flask(__name__)

# route handles and responds with a plain text answer
@app.route('/', methods=['GET'])
def questions():
    questions = "Section-3:" + "\n\n"+ "Expose the results in API for the 10 questions in Section 2 and printing the results as the API response."+"\n\n\n" + "How to open url: localhost:8080/question-1 " + """\n\n\nQuestions: \n\n\n• Retrieve properties that have a price greater than 1 million and are located in "Estados Unidos" (l1).\n\n
• Categorize properties based on their surface area as 'Small' if it's less than 50 square meters, 'Medium' if it's
between 50 and 100 square meters, and 'Large' if it's greater than 100 square meters:\n\n
• List all properties (id) in the "Belgrano" neighborhood (l3) that have the same number of bedrooms and
bathrooms as another property in the dataset:\n\n
• Calculate the average price per square meter (price / surface_total) for each property type (property_type) in
the "Belgrano" neighborhood (l3):\n\n
• Identify properties that have a higher price than the average price of properties with the same number of
bedrooms and bathrooms.\n\n
• Calculate the cumulative price for each property type, ordered by the creation date.\n
• Identify the 10 locations (l3) with the highest total surface area (sum of surface_total) of properties listed for
sale (operation_type = 'Venta'):\n\n
• Find the top 5 most expensive properties (based on price) in the "Palermo" neighborhood (l3) that were listed
in August 2020:\n\n
• Find the top 3 properties with the highest price per square meter (price divided by surface area) within each
property type.\n\n
• Find the top 3 locations (l1, l2, l3) with the highest average price per square meter (price / surface_total) for
properties listed for sale (operation_type = 'Venta') in the year 2020. Exclude locations with fewer than 10
properties listed for sale in 2020 from the results.""" 
    return Response(questions, mimetype='text/plain')    # This line sends a plain text response to flask


@app.route('/question-1', methods=['GET'])
def question_1():
    # Convert the DataFrame to a string and return it
    q1 = "Retrieve properties that have a price greater than 1 million and are located in \"Estados Unidos\" (l1)."
    df1_string = df1.to_string(index=False)    # Converts a DataFrame (df1) into a plain text string without index .
    df1_string = q1 + "\n\n\n" + df1_string
    return Response(df1_string, mimetype='text/plain')    


@app.route('/question-2', methods=['GET'])
def question_2():
    # Convert the DataFrame to a string and return it
    q2 = "Categorize properties based on their surface area as 'Small' if it's less than 50 square meters, 'Medium' if it's between 50 and 100 square meters, and 'Large' if it's greater than 100 square meters."
    df2_string = q2 + "\n\n\n" + "Since the output dataframe is so huge, we are printing first 50 rows of result"+"\n\n\n " + df2.head(50).to_string(index=False)
    return Response(df2_string, mimetype='text/plain')

@app.route('/question-3', methods=['GET'])
def question_3():
    # Convert the DataFrame to a string and return it
    q3 = "• List all properties (id) in the 'Belgrano' neighborhood (l3) that have the same number of bedrooms and bathrooms as another property in the dataset."
    df3_string = q3 + "\n\n\n" + df3.to_string(index=False)
    return Response(df3_string, mimetype='text/plain')

@app.route('/question-4', methods=['GET'])
def question_4():
    # Convert the DataFrame to a string and return it
    q4 = "• Calculate the average price per square meter (price / surface_total) for each property type (property_type) in the 'Belgrano' neighborhood (l3)."
    df4_string = q4 + "\n\n\n" + df4.to_string(index=False)
    return Response(df4_string, mimetype='text/plain')

@app.route('/question-5', methods=['GET'])
def question_5():
    # Convert the DataFrame to a string and return it
    q5 = "• Identify properties that have a higher price than the average price of properties with the same number of bedrooms and bathrooms."
    df5_string = q5 + "\n\n\n" + "Since the output dataframe is so huge, we are printing first 50 rows of result " + "\n\n\n" + df5.head(50).to_string(index=False)
    return Response(df5_string, mimetype='text/plain')

@app.route('/question-6', methods=['GET'])
def question_6():
    # Convert the DataFrame to a string and return it
    q6 = "• Calculate the cumulative price for each property type, ordered by the creation date."
    df6_string = q6 + "\n\n\n" + "Since the output dataframe is so huge, we are printing first 50 rows of result " + "\n\n\n"+ df6.head(50).to_string(index=False)
    return Response(df6_string, mimetype='text/plain')

@app.route('/question-7', methods=['GET'])
def question_7():
    # Convert the DataFrame to a string and return it
    q7 = "• Identify the 10 locations (l3) with the highest total surface area (sum of surface_total) of properties listed for sale (operation_type = 'Venta'):"
    df7_string = q7 + "\n\n\n" + df7.to_string(index=False)
    return Response(df7_string, mimetype='text/plain')

@app.route('/question-8', methods=['GET'])
def question_8():
    # Convert the DataFrame to a string and return it
    q8 = "• Find the top 5 most expensive properties (based on price) in the 'Palermo' neighborhood (l3) that were listed in August 2020:"
    df8_string = q8 + "\n\n\n" + df8.to_string(index=False)
    return Response(df8_string, mimetype='text/plain')

@app.route('/question-9', methods=['GET'])
def question_9():
    # Convert the DataFrame to a string and return it
    q9 = "• Find the top 3 properties with the highest price per square meter (price divided by surface area) within each property type."
    df9_string = q9 + "\n\n\n" + df9.to_string(index=False)
    return Response(df9_string, mimetype='text/plain')

@app.route('/question-10', methods=['GET'])
def question_10():
    # Convert the DataFrame to a string and return it
    q10 = "• Find the top 3 locations (l1, l2, l3) with the highest average price per square meter (price / surface_total) for properties listed for sale (operation_type = 'Venta') in the year 2020. Exclude locations with fewer than 10 properties listed for sale in 2020 from the results."
    df10_string = q10 + "\n\n\n" + df10.to_string(index=False)
    return Response(df10_string, mimetype='text/plain')

# 404 error (page not found). 
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({'error': 'Invalid Question Number'}), 404

# 500 Internal Server Error: This error handler can handle unexpected errors that occur on the server.
@app.errorhandler(500)
def internal_server_error(error):
    return jsonify({'error': 'Internal Server Error'}), 500

@app.route('/shutdown', methods=['GET'])
def shutdown_server():
    print("Shutting down server...")
    app.logger.info("shut")
    os.kill(os.getpid(), signal.SIGINT)  # Send a SIGINT signal to gracefully stop the server
    return 'Server shutting down...'

if __name__ == '__main__':
    print("Server started")
    serve(app,host='127.0.0.1', port=8080)

In [ ]:
# Need to add content related to ngrok though im not using it